In [13]:
import weaviate
from weaviate.classes.query import MetadataQuery
import json
from tqdm import tqdm 
import time

In [3]:
client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
)
collection=client.collections.get("Segments")


/var/folders/q_/nv9cmbv52rx2dhltp0q1l10h0000gn/T/ipykernel_53583/3980344427.py:1: ResourceWarning: unclosed <socket.socket fd=75, family=30, type=1, proto=6, laddr=('::1', 55458, 0, 0), raddr=('::1', 8080, 0, 0)>
  client = weaviate.connect_to_local(


In [4]:

qrels_file_path = 'qrels/qrels.dl23-doc-msmarco-v2.1.txt'
qrels_dict = {}
with open(qrels_file_path, 'r') as file:
    for line in file:
        parts = line.strip().split()
        #count it when it is relevant or not non-relevant
        if (len(parts) >= 3) & (int(parts[-1]) > 0):
            key = parts[0]
            msmarco_text = parts[2]
            if key in qrels_dict:
                qrels_dict[key].append(msmarco_text)
            else:
                qrels_dict[key] = [msmarco_text]

In [5]:
topics_file_path = 'topics/topics.dl23.txt'

topics = {}

with open(topics_file_path, 'r') as file:
    for line in file:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            topics[parts[0]] = parts[1]

In [ ]:
tqdm()

In [14]:
output = {}
for e in tqdm(qrels_dict, desc="Processing queries"):
    output[e] = []
    relevant_doc_num = len(qrels_dict[e])
    topic = topics[e]
    if relevant_doc_num <  30000:
        try:
            response = collection.query.near_text(
                query=topic,
                limit=relevant_doc_num,
                return_metadata=MetadataQuery(distance=True, certainty=True)
            )
        except:
            client.close()
            client.connect()
            response = collection.query.near_text(
                query=topic,
                limit=relevant_doc_num,
                return_metadata=MetadataQuery(distance=True, certainty=True)
            )
        for o in response.objects:
            response_dict = {}
            response_dict['document'] = o.properties
            response_dict['score'] = o.metadata.certainty
            output[e].append(response_dict)


client.close()

# File path where you want to save the JSON
version = topics_file_path.split('.')[-2]
file_path = 'outputs/output_' + version + '.json'

# Open the file in write mode and use json.dump to write the dictionary
with open(file_path, 'w') as file:
    json.dump(output, file, indent=4)

I0000 00:00:1722365852.682344 1868183 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
Processing queries:   4%|▎         | 3/82 [00:27<10:19,  7.84s/it]